# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#!pip install chromadb langchain langchain_community langchain_openai

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [4]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [5]:
llm = OpenAI(temperature=0)

In [6]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [7]:
# !pip install chromadb

In [8]:
import sys
print(sys.executable)


c:\Users\Acer\anaconda3\python.exe


In [9]:
pip show chromadb

Name: chromadb
Version: 1.0.20
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: c:\Users\Acer\anaconda3\Lib\site-packages
Requires: bcrypt, build, grpcio, httpx, importlib-resources, jsonschema, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-sdk, orjson, overrides, posthog, pybase64, pydantic, pypika, pyyaml, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
# import sys
# !{sys.executable} -m pip install chromadb

In [11]:
loader = TextLoader(doc_path, encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [12]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [13]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [15]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [16]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [ ]:
tools = [
    Tool(
        name="sonnets QA System",
        func=sonnets.run,
        description="Useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [19]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\Acer\AppData\Local\Temp\ipykernel_20356\1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [20]:
agent.invoke(
    "What is the most famous sonnet from Shakespeare?"
)



> Entering new AgentExecutor chain...
 The most famous sonnet from Shakespeare is likely to be one of his most well-known works.
Action: sonnets QA System
Action Input: "What is the most famous sonnet from Shakespeare?"
Observation:  The most famous sonnet from Shakespeare is Sonnet 18, also known as "Shall I compare thee to a summer's day?"
Thought: This is a well-known and beloved sonnet, so it makes sense that it would be the most famous.
Final Answer: Sonnet 18, also known as "Shall I compare thee to a summer's day?" is the most famous sonnet from Shakespeare.

> Finished chain.


{'input': 'What is the most famous sonnet from Shakespeare?',
 'output': 'Sonnet 18, also known as "Shall I compare thee to a summer\'s day?" is the most famous sonnet from Shakespeare.'}

In [21]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and type checkers like Mypy. This means that you can use Ruff alongside these tools to get more comprehensive feedback on your code.

3. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. This can save you time and effort when fixing issues in your code.

4. Native implementation of popular Flake8 plugins: Ruff natively

{'input': 'Why use ruff over flake8?',
 'output': 'Ultimately, the decision to use Ruff over Flake8 will depend on your specific needs and preferences. Some users may find that Flake8 meets all of their needs, while others may prefer the larger rule set, better compatibility with other tools, automatic fixing of lint violations, native implementation of popular Flake8 plugins, and Rust-based implementation of Ruff.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [22]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="Useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [23]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.invoke(
    "What is the most famous sonnet from Shakespeare?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What is the most famous sonnet from Shakespeare?"
Observation:  The most famous sonnet from Shakespeare is Sonnet 18, also known as "Shall I compare thee to a summer's day?"


> Finished chain.


{'input': 'What is the most famous sonnet from Shakespeare?',
 'output': ' The most famous sonnet from Shakespeare is Sonnet 18, also known as "Shall I compare thee to a summer\'s day?"'}

In [25]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter, while flake8 is a code analysis tool. They serve different purposes.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.

5. Can be used as a formatter: Ruff can be used as a form

{'input': 'Why use ruff over flake8?',
 'output': '\nThere are a few reasons why someone might choose to use Ruff over Flake8:\n\n1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.\n\n2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.\n\n3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.\n\n4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.\n\n5. Can be used as a formatter: Ruff can be used as a formatter, making it a more versatile tool than Flake8.\n\nUltimately, the choice between Ruff and Flake8 will depend on the specific needs and preferences

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [27]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="Useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [28]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [31]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the shakespeare mention that same tool in one of his sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the Sonnets QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: I should try a different question for the Ruff QA System.
Action: Ruff QA System
Action Input: What is the purpose of ruff?
Observation:  The purpose of Ruff is to provide a linter and formatter for Python code that can be used as a drop-in replacement for Flake8. It also aims to replace other code quality tools such as Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. Ruff is designed to be used alongside a type checker, and it supports any Python version from 3.7 onwards. It is installable under any Python version from 3.7 onwards and does not require the installation of Rust. Ruff's linte

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the shakespeare mention that same tool in one of his sonnets?',
 'output': 'Yes, the tool is mentioned in Sonnet XXXVIII.'}